**-----------VP MWV Avg Analysis---------**

In [1]:
import re
import numpy as np

results = [
    'Total Time - Mean: 939.9850 sec, Std Dev: 1.8880',
    'Total Time - Mean: 934.5850 sec, Std Dev: 2.3122',
    'Total Time - Mean: 970.2700 sec, Std Dev: 3.1113'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 948.2800 sec, Std Dev: 1.4372 sec, CV: 0.15%


-------------------------------

**----------Exhaustive Analysis------------**

In [2]:
import re
import numpy as np

results = [
    'Total Time - Mean: 899.2650 sec, Std Dev: 0.8273',
    'Total Time - Mean: 745.4850 sec, Std Dev: 1.8173',
    'Total Time - Mean: 1915.8250 sec, Std Dev: 4.0093'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 1186.8583 sec, Std Dev: 1.4930 sec, CV: 0.13%


---------------------

**----------VP Log Threshold Analysis-----------**

In [3]:
import re
import numpy as np

results = [
    'Total Time - Mean: 35.6328 sec, Std Dev: 1.1450',
    'Total Time - Mean: 32.3245 sec, Std Dev: 0.3957',
    'Total Time - Mean: 48.1060 sec, Std Dev: 0.4355'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 38.6878 sec, Std Dev: 0.4291 sec, CV: 1.11%


---

**-----------VP Evo Alg-------------**

In [4]:
import re
import numpy as np

results = [
    'Total Time - Mean: 35.4806 sec, Std Dev: 1.0428',
    'Total Time - Mean: 31.4205 sec, Std Dev: 0.5156',
    'Total Time - Mean: 36.0561 sec, Std Dev: 0.6252'
]

means, stds = [], []
for line in results:
    m = re.search(r'Mean:\s*([\d.]+)\s*sec,\s*Std Dev:\s*([\d.]+)', line)
    if not m:
        raise ValueError(f"No match: {line}")
    means.append(float(m.group(1)))
    stds.append(float(m.group(2)))

means = np.array(means, dtype=float)
stds  = np.array(stds,  dtype=float)

k = len(means)

combined_mean = np.mean(means)
combined_std  = np.sqrt(np.sum(stds**2)) / k
cv = (combined_std / combined_mean) * 100 if combined_mean != 0 else float('nan')

print(
    f"Combined Mean: {combined_mean:.4f} sec, "
    f"Std Dev: {combined_std:.4f} sec, CV: {cv:.2f}%"
)

Combined Mean: 34.3191 sec, Std Dev: 0.4402 sec, CV: 1.28%


--------------------

### Comparison between static-log and lamarckian
---

In [5]:
import re
import math
from typing import Tuple, Optional

def parse_stats(s: str) -> Tuple[float, float]:
    mean_match = re.search(r"Mean:\s*([0-9.]+)", s)
    std_match  = re.search(r"Std Dev:\s*([0-9.]+)", s)

    if mean_match is None or std_match is None:
        raise ValueError(f"Could not parse stats from: {s}")

    mean = float(mean_match.group(1))
    std  = float(std_match.group(1))
    return mean, std


def compare_policies(static_stats: str,
                     lamarckian_stats: str,
                     n_static: Optional[int] = None,
                     n_lam: Optional[int] = None):
    """
    Keeps the original outputs, and adds delta_se if run counts are provided.

    delta_std = sqrt(sd_s^2 + sd_l^2)  (SD of difference across random runs)
    delta_se  = sqrt(sd_s^2/n_s + sd_l^2/n_l)  (SE of mean difference)
    """
    mu_s, sigma_s = parse_stats(static_stats)
    mu_l, sigma_l = parse_stats(lamarckian_stats)

    delta = mu_s - mu_l
    sigma_delta = math.sqrt(sigma_s**2 + sigma_l**2)
    normalized = delta / mu_s if mu_s != 0 else float("nan")
    z_score = delta / sigma_delta if sigma_delta > 0 else float("inf")

    out = {
        "delta_mean": delta,
        "delta_std": sigma_delta,
        "normalized_improvement": normalized,
        "z_score": z_score,
    }

    if n_static is not None and n_lam is not None and n_static > 0 and n_lam > 0:
        delta_se = math.sqrt((sigma_s**2)/n_static + (sigma_l**2)/n_lam)
        out["delta_se"] = delta_se

    return out

---

In [7]:
static = "Combined Mean: 38.6878 sec, Std Dev: 0.4291"
lam    = "Combined Mean: 34.3191 sec, Std Dev: 0.4402"

res = compare_policies(static, lam, n_static=3, n_lam=3)
for k, v in res.items():
    print(f"{k}: {v:.4f}")

delta_mean: 4.3687
delta_std: 0.6147
normalized_improvement: 0.1129
z_score: 7.1066
delta_se: 0.3549


---

### Speedup Analysis
---

In [6]:
import re
import math
from typing import Dict, Tuple

def parse_time_stats(s: str) -> Tuple[float, float]:
    """
    Parse strings like:
    'Combined Mean: 29.5141 sec, Std Dev: 0.7580'
    Returns (mean, std_dev)
    """
    mean_match = re.search(r"Mean:\s*([0-9.]+)", s)
    std_match  = re.search(r"Std Dev:\s*([0-9.]+)", s)

    if mean_match is None or std_match is None:
        raise ValueError(f"Could not parse stats from: {s}")

    return float(mean_match.group(1)), float(std_match.group(1))


def compute_speedups(
    stats: Dict[str, str],
    runs: Dict[str, int],
    baseline: str = "Exhaustive",
):
    """
    stats: dict of method -> stats string
    runs:  dict of method -> number of runs
    baseline: method name used as denominator

    Prints speedup S_i = T_baseline / T_i and SE(S_i).
    """
    if baseline not in stats:
        raise ValueError("Baseline method not found in stats.")

    # Parse baseline
    T_exh, sd_exh = parse_time_stats(stats[baseline])
    n_exh = runs[baseline]
    se_exh = sd_exh / math.sqrt(n_exh)

    print(f"Baseline: {baseline}")
    print("-" * 60)
    print(f"{'Method':<15} {'S_i':>10} {'SE(S_i)':>12}")
    print("-" * 60)

    for method, s in stats.items():
        if method == baseline:
            print(f"{method:<15} {1.0:>10.3f} {'-':>12}")
            continue

        T_i, sd_i = parse_time_stats(s)
        n_i = runs[method]
        se_i = sd_i / math.sqrt(n_i)

        # Speedup
        S = T_exh / T_i

        # SE via delta method
        se_S = S * math.sqrt(
            (se_exh / T_exh) ** 2 +
            (se_i   / T_i)   ** 2
        )

        print(f"{method:<15} {S:>10.3f} {se_S:>12.3f}")

---

In [8]:
stats = {
    "VP_MWV":     "Combined Mean: 948.2800 sec, Std Dev: 1.4372",
    "Exhaustive": "Combined Mean: 1186.8583 sec, Std Dev: 1.4930",
    "Log":        "Combined Mean: 38.6878 sec, Std Dev: 0.4291",
    "Lamarckian": "Combined Mean: 34.3191 sec, Std Dev: 0.4402",
}

runs = {
    "VP_MWV": 3,
    "Exhaustive": 3,
    "Log": 3,
    "Lamarckian": 3,
}

compute_speedups(stats, runs)

Baseline: Exhaustive
------------------------------------------------------------
Method                 S_i      SE(S_i)
------------------------------------------------------------
VP_MWV               1.252        0.001
Exhaustive           1.000            -
Log                 30.678        0.198
Lamarckian          34.583        0.257
